In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2758800/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_8_8_13,1.000000,0.683596,1.000000,1.000000,1.000000,6.659821e-09,0.187831,1.389544e-11,1.678114e-08,8.397517e-09,0.000021,0.000082,1.000000,0.000085,167.654346,246.881275,"Hidden Size=[16], regularizer=0.05, learning_r..."
1,model_8_4_13,1.000000,0.683596,1.000000,1.000000,1.000000,6.659821e-09,0.187831,1.123933e-08,3.033294e-08,2.078614e-08,0.000021,0.000082,1.000000,0.000085,167.654346,246.881275,"Hidden Size=[16], regularizer=0.05, learning_r..."
2,model_8_4_5,1.000000,0.683596,1.000000,1.000000,1.000000,6.659821e-09,0.187831,1.123933e-08,3.033294e-08,2.078614e-08,0.000021,0.000082,1.000000,0.000085,167.654346,246.881275,"Hidden Size=[16], regularizer=0.05, learning_r..."
3,model_8_4_6,1.000000,0.683596,1.000000,1.000000,1.000000,6.659821e-09,0.187831,1.123933e-08,3.033294e-08,2.078614e-08,0.000021,0.000082,1.000000,0.000085,167.654346,246.881275,"Hidden Size=[16], regularizer=0.05, learning_r..."
4,model_8_5_0,1.000000,0.683596,1.000000,1.000000,1.000000,6.659821e-09,0.187831,1.121908e-08,7.616776e-09,9.417926e-09,0.000021,0.000082,1.000000,0.000085,167.654346,246.881275,"Hidden Size=[16], regularizer=0.05, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3594,model_18_1_6,0.992252,0.657430,0.996608,0.996365,0.996755,4.599611e-03,0.203364,3.430596e-03,1.758384e-03,2.594490e-03,0.033387,0.067820,1.003509,0.070708,164.763567,258.617006,"Hidden Size=[19], regularizer=0.2, learning_ra..."
3682,model_18_1_5,0.990777,0.652420,0.996682,0.996317,0.996788,5.474985e-03,0.206338,3.355364e-03,1.781821e-03,2.568593e-03,0.035304,0.073993,1.004176,0.077143,164.415131,258.268570,"Hidden Size=[19], regularizer=0.2, learning_ra..."
3765,model_18_1_4,0.988952,0.646739,0.996762,0.996244,0.996817,6.558689e-03,0.209711,3.274113e-03,1.816908e-03,2.545511e-03,0.037454,0.080986,1.005003,0.084433,164.053929,257.907368,"Hidden Size=[19], regularizer=0.2, learning_ra..."
3850,model_18_1_3,0.986692,0.640275,0.996848,0.996144,0.996841,7.899955e-03,0.213548,3.187184e-03,1.865448e-03,2.526315e-03,0.039838,0.088882,1.006026,0.092666,163.681796,257.535235,"Hidden Size=[19], regularizer=0.2, learning_ra..."
